# Optimización Cartera de Préstamos a Titulizar

In [ ]:
from IPython.core.display import HTML
display(HTML("<style>pre { white-space: pre !important; }</style>"))
from pyspark.sql import functions as F, DataFrame
import datetime as dt
from datetime import date, datetime, timedelta
from dateutil.relativedelta import relativedelta
from pyspark.sql.window import Window
import pyspark.sql.types as t
from decimal import Decimal
from pyspark.sql.functions import regexp_replace
import itertools

In [ ]:
from dataproc_sdk.dataproc_sdk_datiopysparksession.datiopysparksession import DatioPysparkSession
datioSparkSession = DatioPysparkSession().get_or_create()

from dataproc_sdk.dataproc_sdk_datiopysparksession import datiopysparksession
dataproc = datiopysparksession.DatioPysparkSession().get_or_create()

from dataproc_sdk.dataproc_sdk_schema.datioschema import DatioSchema

## Instalamos librerías

In [ ]:
path_csv = '/data/sandboxes/dslb/data/Joystick/TITULIZACIONES/modelo_optimizacion/SRT_Model_Optimizador-GPT.csv'
path_csv ='/data/sandboxes/dslb/data/Joystick/TITULIZACIONES/modelo_optimizacion'

In [ ]:
# SRT_Model_Optimizador-GPT.csv

## Leemos CSV con la Cartera de Préstamos

In [ ]:
df = spark.read.option('header','True').option('delimiter',',').csv(path_csv)
df.show(5,False)

In [ ]:
df_p = df.withColumn('Importe del Expediente',F.col('Importe del Expediente').cast('float')
        ).withColumn('CR_Porcentaje',F.col('CR_Porcentaje').cast('float'))

In [ ]:
df_p.printSchema()

In [ ]:
df_p.show(5,False)

In [ ]:
df_p.where(F.col('Importe del Expediente') >= 0).show(5,False)

In [ ]:
import pandas as pd
import numpy as np

# Asegurándonos de que los datos están en el formato correcto
raw_data = df.toPandas() #pd.read_csv(path_csv)
raw_data.columns = ['Nº Expediente', 'Tramo', 'País', 'Clase Producto Financiero', 'Divisa Base', 'Importe del Expediente', 'CR_Porcentaje']


In [ ]:
# raw_data.info(verbose = True, null_counts = False)

In [ ]:
raw_data['Nº Expediente'] = raw_data['Nº Expediente'].astype('str') 
raw_data['Tramo'] = raw_data['Tramo'].astype('str') 
raw_data['País'] = raw_data['País'].astype('str') 
raw_data['Clase Producto Financiero'] = raw_data['Clase Producto Financiero'].astype('str') 
raw_data['Divisa Base'] = raw_data['Divisa Base'].astype('str') 
raw_data['Importe del Expediente'] = raw_data['Importe del Expediente'].astype(float)
raw_data['CR_Porcentaje'] = raw_data['CR_Porcentaje'].astype(float)

## Preparamos los datos

In [ ]:
#Eliminamos registros con Importes nan y que sean cero.
data = raw_data.dropna(subset=['Importe del Expediente'])
data = data[data['Importe del Expediente'] >= 0]

# Ordenar los datos por porcentaje de riesgo descendente
df = data.sort_values(by='CR_Porcentaje', ascending=False).reset_index(drop=True)

# Convertir datos a listas para su uso en Pulp y OR-Tools
importes = data['Importe del Expediente'].tolist()
riesgos = data['CR_Porcentaje'].tolist()
divisas = data['Divisa Base'].tolist()
paises = data['País'].tolist()
ranking = data.index.values.tolist()
n = len(importes)


# Resolución sin programación lineal

In [ ]:
import pandas as pd

# Inicializar las restricciones
total_importe = 0
divisa_importe = {divisa: 0 for divisa in df['Divisa Base'].unique()}
pais_importe = {pais: 0 for pais in df['País'].unique()}

# Lista para almacenar los índices de los expedientes seleccionados y sus proporciones
selected_expedients = []

# Agregar expedientes uno por uno
for i in df.index:
    expediente_importe = df.loc[i, 'Importe del Expediente']
    divisa = df.loc[i, 'Divisa Base']
    pais = df.loc[i, 'País']

    max_proportion = 1.0
    if total_importe + expediente_importe > 230000000:
        max_proportion = (230000000 - total_importe) / expediente_importe
    if divisa_importe[divisa] + expediente_importe > 75000000:
        max_proportion = min(max_proportion, (75000000 - divisa_importe[divisa]) / expediente_importe)
    if pais_importe[pais] + expediente_importe > 75000000:
        max_proportion = min(max_proportion, (75000000 - pais_importe[pais]) / expediente_importe)

    if max_proportion > 0:
        # Agregar el expediente y la proporción seleccionada
        selected_expedients.append((i, max_proportion))

        # Actualizar las restricciones
        total_importe += expediente_importe * max_proportion
        divisa_importe[divisa] += expediente_importe * max_proportion
        pais_importe[pais] += expediente_importe * max_proportion

        # Terminar si llegamos al límite total
        if total_importe >= 230000000:
            break

# Obtener los resultados
result_data = []
for i, proportion in selected_expedients:
    row = df.loc[i].copy()
    row['porcentaje optimo'] = proportion
    result_data.append(row)

result_df = pd.DataFrame(result_data)

result_df['importe optimo'] = result_df['porcentaje optimo'] * result_df['Importe del Expediente']

print(result_df['CR_Porcentaje'].sum())

result_df


In [ ]:
grouped_data = result_df.groupby('Divisa Base')['importe optimo'].sum().reset_index()
print(grouped_data['importe optimo'].sum())
grouped_data

In [ ]:
grouped_data = result_df.groupby('País')['importe optimo'].sum().reset_index()
print(grouped_data['importe optimo'].sum())
grouped_data